In [1]:
import ipywidgets as widgets
from abtem import PixelatedDetector, FrozenPhonons, Probe, FlexibleAnnularDetector, show_atoms, SMatrix
from abtem.measure import Measurement
from abtem.visualize.interactive.artists import MeasurementArtist2d
from abtem.visualize.interactive.canvas import Canvas
from abtem.visualize.interactive.tools import SelectPixelTool, SelectAnnularRadiiTool
from ase.io import read
from traitlets import link

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [108]:
# from ase.cluster import Decahedron
# import numpy as np
# from abtem import GridScan, Potential
# from ase import build

# atoms = Decahedron('Au', 4, 2, 1, latticeconstant=None)
# atoms.rotate(30, 'x', center='cop')
# atoms.center(vacuum=2)

# carbon = build.bulk('C', 'diamond', cubic=True)
# carbon *= (int(atoms.cell[0,0]/carbon.cell[0,0]),int(atoms.cell[1,1]/carbon.cell[1,1]),4)

# carbon.set_cell((atoms.cell[0,0], atoms.cell[1,1], carbon.cell[2,2]), scale_atoms=True)
# carbon.positions[:] += np.random.randn(len(carbon), 3) * .3
# carbon.positions[:,2] += atoms.positions[:,2].max()

# #carbon.wrap()

# #atoms += carbon

# atoms.center(axis=2, vacuum=2)

# show_atoms(atoms, plane='xy')

In [109]:
# atoms = read('srtio3_100.cif')
# atoms *= (6, 6, 20)

# # show_atoms(atoms)

In [110]:
# potential = Potential(atoms=FrozenPhonons(atoms, sigmas=.0, num_configs=1),
#                       projection='infinite',
#                       parametrization='kirkland')

# #S = Probe(semiangle_cutoff=20, energy=100e3, gpts=512, device='gpu', rolloff=2)
# S = SMatrix(semiangle_cutoff=20, energy=100e3, gpts=512, device='gpu', rolloff=2)

# detector1 = FlexibleAnnularDetector()
# detector2 = PixelatedDetector()

# scan = GridScan((0, 0), np.diag(atoms.cell)[:2] / 6, sampling=S.ctf.nyquist_sampling * .8)

# measurement1, measurement2 = S.scan(scan, [detector1, detector2], potential, max_batch_probes=500)

In [111]:
# potential = Potential(atoms=FrozenPhonons(atoms, sigmas=.08, num_configs=16),
#                       projection='infinite',
#                       parametrization='kirkland')

# #S = Probe(semiangle_cutoff=20, energy=100e3, gpts=512, device='gpu', rolloff=2)
# S = SMatrix(semiangle_cutoff=20, energy=100e3, gpts=512, device='gpu', rolloff=2)

# detector1 = FlexibleAnnularDetector()
# detector2 = PixelatedDetector()

# scan = GridScan((0, 0), np.diag(atoms.cell)[:2] / 6, sampling=S.ctf.nyquist_sampling * .8)

# tds_measurement1, tds_measurement2 = S.scan(scan, [detector1, detector2], potential, max_batch_probes=500)

In [113]:
#measurement1.write('adf.hdf5')
#measurement2.write('diffraction.hdf5')

In [112]:
#tds_measurement1.write('tds_adf.hdf5')
#tds_measurement2.write('tds_diffraction.hdf5')

In [2]:
measurement1 = Measurement.read('data/adf.hdf5')
measurement2 = Measurement.read('data/diffraction.hdf5')
tds_measurement1 = Measurement.read('data/tds_adf.hdf5')
tds_measurement2 = Measurement.read('data/tds_diffraction.hdf5')

In [3]:
canvas1 = Canvas()
canvas2 = Canvas()

select = SelectAnnularRadiiTool(min_width=1, max_radius=160)
select.inner_radius = 50
select.outer_radius = 150

image_artist1 = MeasurementArtist2d()
image_artist2 = MeasurementArtist2d()
canvas1.artists = {'image_artist': image_artist1}
canvas2.artists = {'image_artist': image_artist2}

toggle_log_button = widgets.ToggleButton(description='Log scale', value=False)
position_tool = SelectPixelTool(image_artist=image_artist1.image_artist, marker=True)

tds_button = widgets.ToggleButton(description='TDS', value=True)


def update_image(*args):
    if tds_button.value:
        new_measurement = tds_measurement1.integrate(select.inner_radius, select.outer_radius)
    else:
        new_measurement = measurement1.integrate(select.inner_radius, select.outer_radius)
    image_artist1.measurement = None
    #print(image_artist1.image_artist.autoadjust_colorscale)
    image_artist1.measurement = new_measurement.tile((6, 6)).gaussian_filter((.5, .5))#.interpolate(.2)


def update_diffraction_pattern(*args):
    index_x = position_tool.indices[0] % measurement2.shape[0]
    index_y = position_tool.indices[1] % measurement2.shape[1]
    
    if tds_button.value:
        new_meausurement = tds_measurement2[index_x, index_y].copy()
    else:
        new_meausurement = measurement2[index_x, index_y].copy()
    
    if toggle_log_button.value == True:
        new_meausurement.array[:] = np.log(new_meausurement.array[:])

    image_artist2.measurement = new_meausurement


select.activate(canvas2)
select.observe(update_image, ('inner_radius', 'outer_radius'))

canvas1.title = 'Image'
canvas2.title = 'Diffraction pattern'

position_tool.observe(update_diffraction_pattern, 'indices')
position_tool.activate(canvas1)

update_image()
update_diffraction_pattern()

canvas1.adjust_labels_to_artists()
canvas1.adjust_limits_to_artists()
canvas2.adjust_labels_to_artists()
canvas2.adjust_limits_to_artists()

inner_slider = widgets.FloatRangeSlider(description='Limits', min=0, max=160, value=[50, 60], step=1)
toggle_log_button.observe(update_diffraction_pattern, 'value')

link((select, 'inner_radius'), (inner_slider, 'value'),
     (lambda x: [x, select.outer_radius], lambda x: min(x[0], select.outer_radius - 1)))
link((select, 'outer_radius'), (inner_slider, 'value'),
     (lambda x: [select.inner_radius, x], lambda x: max(x[1], select.inner_radius + 1)))

autoadjust_button = widgets.ToggleButton(description='Autoadjust colorscale', value=True)

link((autoadjust_button, 'value'), (image_artist1.image_artist, 'autoadjust_colorscale'))
link((autoadjust_button, 'value'), (image_artist2.image_artist, 'autoadjust_colorscale'))

autoadjust_button.observe(update_image, 'value')
autoadjust_button.observe(update_diffraction_pattern, 'value')

canvas2.x_label = 'Scattering angle x [mrad.]'
canvas2.y_label = 'Scattering angle y [mrad.]'

tds_button.observe(update_image, 'value')
tds_button.observe(update_diffraction_pattern, 'value')

whitespace = widgets.HBox([])
whitespace.layout.height = '30px'

widgets.HBox(
    [canvas2.widget, canvas1.widget, widgets.VBox([whitespace, inner_slider, 
                                                   image_artist2.image_artist.power_scale_slider, widgets.HBox([autoadjust_button, tds_button])])])